In [1]:
import pandas as pd
#from minority_report.clean_data import CleanData

In [2]:
df = pd.read_csv('../raw_data/data.csv')

/Users/camillebenhaim/.pyenv/versions/3.8.5/envs/minority_report/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df = df[:1_000_000]

In [ ]:
#df.head()

In [ ]:
# mask booleans
#not_unknown = df['precinct_number'] != -99.0
#not_nan = df['precinct_number'] == df['precinct_number']
#df = df[not_unknown & not_nan]
#df = df[df['time'] == df['time']]

In [ ]:
#df.shape

In [ ]:
#df.isnull().sum()

## Get only weird precints

In [ ]:
precinct_numbers = np.sort(df['precinct_number'].unique())

In [ ]:
len(df['precinct_number'].unique())

In [ ]:
# nb rows with weird precinct
for element in df['precinct_number']:
    if element == -99. or element != element:
        liste.append(element)
len(liste)

In [ ]:
# rows to drop : no precinct, no coordinates
data = df[unknown | nane]
#test
data.shape[0] == 2916

In [ ]:
to_drop = data[data['coordinates'] != data['coordinates']]
to_drop.shape

## Keep only exploitable rows

In [ ]:
#rows to keep : no precinct but coordinates
to_keep = data[data['coordinates'] == data['coordinates']]
to_keep.shape

In [ ]:
to_keep['coordinates'].unique()

In [ ]:
to_keep['date'].unique()

## Open the precinct map data

In [ ]:
# open precinct distribution file
import json
with open('../raw_data/precincts.geojson') as f:
    geo = json.load(f)
geo

In [ ]:
#test
len(geo['features']) == 77

In [ ]:
p1 = geo['features'][0]
p1['geometry']['coordinates']

In [ ]:
p1_array = np.array(p1['geometry']['coordinates'], dtype=object)
p1_array.shape

In [ ]:
p1_array[0][0]

In [ ]:
for precinct in geo['features']:
    

## Create default_values per precinct

### Essais

In [ ]:
df.head()

In [ ]:
geo = df[df['precinct_number'] == 1][['precinct_number','latitude', 'longitude']]
geo.head()

In [ ]:
geo.describe()

In [ ]:
med_lat_1 = geo['latitude'].median()
med_lon_1 = geo['longitude'].median()

In [ ]:
np.sort(df['precinct_number'].unique())

In [ ]:
#test
#df.shape == (6980243, 22)
#type(df) == pd.DataFrame

In [ ]:
data_base = drop_miss(df)

In [ ]:
data = miss_lon_lat(data_base[:100000])

In [ ]:
data.isnull().sum()

In [ ]:
geo = df[df['precinct_number'] == 1][['latitude', 'longitude']]
geo

In [ ]:
values = {'latitude': geo['latitude'].median(), 'longitude':geo['longitude'].median()}
values

In [ ]:
df[df['precinct_number']==1].fillna(value=values) 

In [ ]:
_.isnull().sum()

### Functions

In [4]:
def drop_miss (df):
    '''drop precinct and time missing values'''
    
    #drop precinct missing values
    not_unknown = df['precinct_number'] != -99.0
    not_nan = df['precinct_number'] == df['precinct_number']
    df = df[not_unknown & not_nan]
    
    #drop offense_type, time and date missing values
    df = df[df['time'] == df['time']]
    df = df[df['date'] == df['date']]
    df = df[df['offense_type'] == df['offense_type']]
    df = df[df['crime_completed'] == df['crime_completed']]
    
    return df

In [5]:
def miss_lon_lat (df):
    '''replace latitude and longitude missing values with median values by precinct'''
    
    data = df.copy()
    for precinct in np.sort(data['precinct_number'].unique()):
        #print(f'Modifying precinct{precinct}')
        geo = data[data['precinct_number'] == precinct][['latitude', 'longitude']]
        # get median crime location for each precinct
        values = {'latitude': geo['latitude'].median(), 'longitude':geo['longitude'].median()}
        # fillna lon and lat column for each crime depending on its precinct
        data[data['precinct_number']==precinct] = data[data['precinct_number']==precinct].fillna(value=values) 
    return data
    

In [6]:
def removing_date_before_2007(df):
        '''
        Remove any dates before 2007 & values not corresponding to our US date format month/day/year, returns pd.series
        '''
        rg_expression = r'(1[0-2]|0?[1-9])\/(3[01]|[12][0-9]|0?[1-9])\/20([1-2][0-9]|[0-2][7-9])'
        boolean_values = df['date'].str.match(rg_expression)
        return df[boolean_values]

## Handle victim missing values

### Essais

In [ ]:
df= drop_miss(df)

In [ ]:
df = removing_date_before_2007(df)

In [ ]:
df = miss_lon_lat(df)

In [ ]:
df.isnull().sum()

In [ ]:
data['victim_sex'].unique()

In [ ]:


data['victim_age_done'] = [element if element in liste else 'UNKNOWN' for element in data['victim_age']]

In [ ]:
data['victim_age_done'].unique()

### Function

In [7]:
def miss_victim (df):
    data = df.copy()
    age_liste = ['<18', '45-64', '18-24', '25-44']
    race_liste = ['BLACK', 'WHITE', 'UNKNOWN', 'WHITE HISPANIC', 'BLACK HISPANIC',
       'ASIAN / PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKAN NATIVE']
    sex_liste = ['M', 'F', 'D', 'E']
    data['victim_age'] = [element if element in age_liste else 'UNKNOWN' for element in data['victim_age']]
    data['victim_race'] = [element if element in race_liste else 'UNKNOWN' for element in data['victim_race']]
    data['victim_sex'] = [element if element in sex_liste else 'UNKNOWN' for element in data['victim_sex']]
    return data

In [8]:
def round_int(series):
        """this functions rounds pd.series of int, up"""
        result = [round(x) for x in series]
        return result
def complete_to_boolean(series):
        """turns complete/incomplete into boolean value"""
        result = series.replace({'COMPLETED': True, 'INCOMPLETE': False})
        return result

## Replace missing borough 

### Essais

In [9]:
df = drop_miss(df)

In [10]:
df = removing_date_before_2007(df)

In [11]:
df = miss_lon_lat(df)

In [12]:
df = miss_victim(df)

In [13]:
df['precinct_number'] = round_int(df['precinct_number'])
df['crime_completed'] = complete_to_boolean(df['crime_completed'])

In [ ]:
df.isnull().sum()

In [ ]:
df['precinct_number'].value_counts()

### Function

In [14]:
def miss_borough (df):
    '''replace borough correct values depending on the precinct'''
    
    data = df.copy()
    data['new_borough'] = [
        'MANHATTAN' if precinct <=34
        else 'BRONX' if precinct <=54
        else 'BROOKLYN' if precinct <= 94
        else 'QUEENS' if precinct <= 115
        else 'STATEN ISLAND'
        for precinct in data['precinct_number']
    ]
    
   
    data.drop(columns='borough', inplace=True)
    data.rename(columns={'new_borough':'borough'},inplace=True)
    return data

### Tests

In [15]:
df = miss_borough(df)

In [ ]:
df.columns

In [ ]:
df['borough'].unique()

In [ ]:
for element in np.sort(data['precinct_number'].unique()):
    print(element, data[data['precinct_number']==element]['new_borough'].value_counts())

In [ ]:
df.isnull().sum()

## handle missing patrol_borough value

In [ ]:
df['patrol_borough'].unique()

In [ ]:
df['patrol_borough'].value_counts()

In [ ]:
df[['patrol_borough', 'borough']].head()

In [ ]:
to_decide = df[df['patrol_borough'] !=df['patrol_borough']][['date', 'borough', 'offense_type', 'longitude', 'latitude']]
to_decide

In [ ]:
to_decide['offense_type'].value_counts()

In [ ]:
test = pd.DataFrame(df['offense_type'].value_counts()).reset_index()
test.rename(columns={'index':'offense', 'offense_type':'nb'}, inplace=True)

In [ ]:
test[test['offense'] == 'MURDER & NON-NEGL. MANSLAUGHTER']

In [16]:
def miss_patrol_borough (df):
    '''replace borough correct values depending on the precinct'''
    
    data = df.copy()
    bronx = [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52]
    bklyn_south = [60, 61, 62, 63, 66, 67, 68, 69, 70, 71, 72, 76, 78]
    bklyn_north = [73, 75, 77, 79, 81, 83, 84, 88, 90, 94]
    man_south = [1, 5, 6, 7, 9, 10, 13, 14, 17, 18]
    man_north = [19, 20, 22, 23, 24, 25, 26, 28, 30, 32, 33, 34]
    queens_south = [100, 101, 102, 103, 105, 106, 107, 113]
    queens_north = [104, 108, 109, 110, 111, 112, 114, 115]
    staten = [120, 121, 122, 123]
    data['new_patrol'] = [
        'PATROL BORO BRONX' if precinct in bronx
        else 'PATROL BORO BKLYN SOUTH' if precinct in bklyn_south
        else 'PATROL BORO BKLYN NORTH' if precinct in bklyn_north
        else 'PATROL BORO MAN SOUTH' if precinct in man_south
        else 'PATROL BORO MAN NORTH' if precinct in man_north
        else 'PATROL BORO QUEENS SOUTH' if precinct in queens_south
        else 'PATROL BORO QUEENS NORTH' if precinct in queens_north
        else 'PATROL BORO STATEN ISLAND'       
        for precinct in data['precinct_number']
    ]
    
    data.drop(columns='patrol_borough', inplace=True)
    data.rename(columns={'new_patrol':'patrol_borough'},inplace=True)
    return data

In [ ]:
bronx = [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52]
bklyn_south = [60, 61, 62, 63, 66, 67, 68, 69, 70, 71, 72, 76, 78]
bklyn_north = [73, 75, 77, 79, 81, 83, 84, 88, 90, 94]
man_south = [1, 5, 6, 7, 9, 10, 13, 14, 17, 18]
man_north = [19, 20, 22, 23, 24, 25, 26, 28, 30, 32, 33, 34]
queens_south = [100, 101, 102, 103, 105, 106, 107, 113]
queens_north = [104, 108, 109, 110, 111, 112, 114, 115]
staten = [120, 121, 122, 123]
all_patrol = [bronx,bklyn_south ,bklyn_north, man_south,man_north,queens_south, queens_north,staten]

In [ ]:
verified = []
for element in all_patrol:
    for precinct in element:
        if precinct not in precinct_numbers:
            print (precinct, 'alert')
        else:
            verified.append(precinct)

In [ ]:
np.sort(np.array(verified)) == precinct_numbers

In [17]:
df = miss_patrol_borough(df)

In [ ]:
df['patrol_borough'].isnull().sum()

In [ ]:
df['patrol_borough'].unique()

In [18]:
df.isnull().sum()

complaint_id            0
date                    0
time                    0
precinct_number         0
offense_type            0
crime_completed         0
offense_level           0
premise_desc       188862
premise              4258
jurisdiction            0
park_name          821902
suspect_age        707427
suspect_race       489659
suspect_sex        522415
latitude                0
longitude               0
metro              805092
victim_age              0
victim_race             0
victim_sex              0
borough                 0
patrol_borough          0
dtype: int64